In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout

import cv2
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline 
LABELS = ['CAT', 'DOG']

X = []
print("Reading data...")
for directory in LABELS:
    path = os.path.join(directory, "resized")
    for img_name in os.listdir(path):
        img = cv2.imread(os.path.join(path, img_name), cv2.IMREAD_GRAYSCALE)
        X.append([np.array(img), np.eye(2)[LABELS.index(directory)]])

print("Rearranging data...")
np.random.shuffle(X)
x_train = []
y_train = []
for img in X:
    x_train.append(img[0])
    y_train.append(img[1])
if len(x_train) == len(y_train):
    slice_index = int(len(x_train)*0.2)
    x_val = x_train[:slice_index]
    y_val = y_train[:slice_index]
    x_train = x_train[slice_index:]
    y_train = y_train[slice_index:]
    
    x_train = np.array(x_train)
    x_val = np.array(x_val)
    x_train = x_train.reshape(x_train.shape[0], 75, 75, 1)
    x_val = x_val.reshape(x_val.shape[0], 75, 75, 1)
    
    #normalize image data
    x_train = np.true_divide(x_train, 255)
    x_val = np.true_divide(x_val, 255)
    
    y_train = np.array(y_train)
    y_val = np.array(y_val)
        
    
    print(f"completed: {len(X)} items added to training data")
    print(f"x_train: {len(x_train)}  |  {x_train.shape}")
    print(f"y_train: {len(y_train)}")
    print(f"x_val: {len(x_val)}")
    print(f"y_val: {len(y_val)}")
else:
    print("something went wrong with the data")


Reading data...
Rearranging data...
completed: 25004 items added to training data
x_train: 20004  |  (20004, 75, 75, 1)
y_train: 20004
x_val: 5000
y_val: 5000


In [13]:
from PIL import Image
print(y_val[0])
print(x_train[0].dtype)
print(dir(x_train[0]))
#img = Image.fromarray(x_train[1], mode='L')
#cv2.imshow()

[0. 1.]
float64
['T', '__abs__', '__add__', '__and__', '__array__', '__array_finalize__', '__array_function__', '__array_interface__', '__array_prepare__', '__array_priority__', '__array_struct__', '__array_ufunc__', '__array_wrap__', '__bool__', '__class__', '__complex__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__ilshift__', '__imatmul__', '__imod__', '__imul__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__irshift__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__lshift__', '__lt__', '__matmul__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv

In [15]:
EPOCHS = 4
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), activation='relu', input_shape=(75,75,1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(LABELS), activation='relu'))

#train
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy', 'categorical_crossentropy'])
print('compiled.')
model.summary()
history = model.fit(x_train, y_train, batch_size=25, epochs=EPOCHS, verbose=1, validation_data=(x_val, y_val))

W1102 19:57:57.045602 20568 training_utils.py:1210] When passing input data as arrays, do not specify `steps_per_epoch`/`steps` argument. Please use `batch_size` instead.


compiled.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 73, 73, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 36, 36, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0         
_________________________________________________________________
dense (Dense)                (None, 128)      

W1102 19:57:57.644145 20568 deprecation.py:323] From C:\Users\Jaycop\Miniconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\ops\math_grad.py:1393: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1102 19:57:57.733226 20568 deprecation.py:323] From C:\Users\Jaycop\Miniconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\keras\optimizer_v2\optimizer_v2.py:466: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


20004/20004 [==============================] - 348s 17ms/sample - loss: nan - accuracy: 0.5024 - categorical_crossentropy: nan - val_loss: nan - val_accuracy: 0.4904 - val_categorical_crossentropy: nan
Epoch 2/4
20004/20004 [==============================] - 344s 17ms/sample - loss: nan - accuracy: 0.5024 - categorical_crossentropy: nan - val_loss: nan - val_accuracy: 0.4904 - val_categorical_crossentropy: nan
Epoch 3/4
20004/20004 [==============================] - 347s 17ms/sample - loss: nan - accuracy: 0.5024 - categorical_crossentropy: nan - val_loss: nan - val_accuracy: 0.4904 - val_categorical_crossentropy: nan
Epoch 4/4
20004/20004 [==============================] - 354s 18ms/sample - loss: nan - accuracy: 0.5024 - categorical_crossentropy: nan - val_loss: nan - val_accuracy: 0.4904 - val_categorical_crossentropy: nan


In [27]:
"""from matplotlib import pyplot
%matplotlib inline
pyplot.plot(history.history["accuracy"])

pyplot.show()
print(history.history['accuracy'])
"""
import h5py
import time
h5 =  h5py.File(f"catvsdog-{int(time.time())}.hdf5", "w")
print(dir(h5))
print(history.history)

['_MutableMapping__marker', '__abstractmethods__', '__bool__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_d', '_e', '_gcpl_crt_order', '_id', '_lapl', '_lcpl', '_swmr_mode', 'attrs', 'clear', 'close', 'copy', 'create_dataset', 'create_dataset_like', 'create_group', 'create_virtual_dataset', 'driver', 'fid', 'file', 'filename', 'flush', 'get', 'id', 'items', 'keys', 'libver', 'mode', 'move', 'name', 'parent', 'pop', 'popitem', 'ref', 'regionref', 'require_